In [3]:
import cv2
import random
import time
import mediapipe as mp
import numpy as np
import uuid, os

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

<img src= 'https://camo.githubusercontent.com/acd97b0df9f2fe9d61518cf45ad896df855f6a98/68747470733a2f2f692e696d6775722e636f6d2f717052414365722e706e67'/>

In [ ]:
#0 for rock, 1 for paper, 2 for scissors 
score = 0
c_score = 0   
draw = None
def janken(computer, sign):
    global score
    global c_score
    global draw

    if computer == sign:
        draw = 'thats a draw, try again'

    if computer == 0:
        if sign == 1:
            score+=1
            draw= 'You win this round'
        elif sign ==2:
            draw= 'computer wins this round'
            c_score+=1

    elif computer == 1:
        if sign == 2:
            draw= 'You win this round'
            score+=1
        elif sign == 0:
            draw= 'computer wins this round'
            c_score+=1

    if computer == 2:
        if sign == 0:
            draw= 'you win this round'
            score+=1
        elif sign == 1:
            draw= 'computer wins this round'
            c_score+=1
    return sign, score, c_score, draw


In [ ]:
janken(0,2)

(2, 0, 1, 'computer wins this round')

In [ ]:
# initiate video session
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=1) as hands: 
    frame_rate = 1
    prev = 0
    while cap.isOpened():

        time_elapsed = time.time() - prev
        ret, frame = cap.read()
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            # Detections
            results = hands.process(image)

            # Set flag to true
            image.flags.writeable = True

            # RGB 2 BGR
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Detections
            print(results)

            # Rendering results
            if results.multi_hand_landmarks:

                computer= 2 # random.choice([0,1,2])

                for num, hand in enumerate(results.multi_hand_landmarks):
                    # Changing colors and add connections (bones) and joints 
                    mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                             )
                    index_down = - results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y
                    index_up = - results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y

                    middle_finger_down = - results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y
                    middle_finger_up = - results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y

                    ring_finger_down = - results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_MCP].y
                    ring_finger_up = - results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y

                    pinky_down = - results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_MCP].y
                    pinky_up = - results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_TIP].y

                    THUMB_down = - results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_CMC].y
                    THUMB_up = - results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_TIP].y

                    if index_down-index_up< 0 and middle_finger_down - middle_finger_up<0 and THUMB_down- THUMB_up<0 and ring_finger_down - ring_finger_up>0 and pinky_down- pinky_up>0:
                        sign, score, c_score, draw =janken(computer,2)
                        print(sign, score, c_score, draw)
                        cv2.putText(image, 'SCORE', (15,12), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,255 ,0), 1, cv2.LINE_AA)
                        cv2.putText(image, 'Computer:', 
                                                    (10,60), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                        cv2.putText(image, str(c_score), 
                                                    (180,60), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                        cv2.putText(image, 'Player', 
                                                    (10,90), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)       
                        cv2.putText(image, str(score), 
                                                    (180,90), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                        cv2.putText(image, str(draw), 
                                                    (10,120), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
                    elif index_down - index_up<0 and middle_finger_down -  middle_finger_up<0 and THUMB_down - THUMB_up<0 and ring_finger_down - ring_finger_up <0 and pinky_down - pinky_up<0:
                        sign, score, c_score, draw =janken(computer,1)
                        print(sign, score, c_score, draw)
                        cv2.putText(image, 'SCORE', (15,12), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                        cv2.putText(image, 'Computer:', 
                                                    (10,60), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                        cv2.putText(image, str(c_score), 
                                                    (180,60), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                        cv2.putText(image, 'Player', 
                                                    (10,90), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)       
                        cv2.putText(image, str(score), 
                                                    (180,90), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                        cv2.putText(image, str(draw), 
                                                    (10,120), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
                    elif index_down- index_up>0 and middle_finger_down  - middle_finger_up>0 and ring_finger_down - ring_finger_up > 0 and pinky_down - pinky_up>0:
                        sign, score, c_score, draw =janken(computer,0)
                        print(sign, score, c_score, draw)
                        cv2.putText(image, 'SCORE', (15,12), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                        cv2.putText(image, 'Computer:', 
                                                    (10,60), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                        cv2.putText(image, str(c_score), 
                                                    (180,60), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                        cv2.putText(image, 'Player', 
                                                    (10,90), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)       
                        cv2.putText(image, str(score), 
                                                    (180,90), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA) 
                        cv2.putText(image, str(draw), 
                                                    (10,120), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA) 
                    else:
                        print('wrong movement')
                        cv2.putText(image, 'SCORE', (15,12), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                        cv2.putText(image, 'Computer:', 
                                                    (10,60), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                        cv2.putText(image, str('-'), 
                                                    (180,60), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                        cv2.putText(image, 'Player', 
                                                    (10,90), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)       
                        cv2.putText(image, str('-'), 
                                                    (180,90), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)     
            
                        cv2.putText(image, 'wrong movement', 
                                                    (10,120), 
                                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA) 
            
            cv2.putText(image, 'SCORE', (15,12), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, 'Computer:', 
                                        (10,60), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str('-'), 
                                        (180,60), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, 'Player', 
                                        (10,90), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)       
            cv2.putText(image, str('-'), 
                                        (180,90), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)       
            #for hand in results:
            cv2.imshow('Hand Tracking', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0 1 1 you win this round
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0 2 1 you win this round
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
2 2 1 thats a draw, try again
<class '

KeyboardInterrupt: 

In [10]:
#for hand_landmarks in results.multi_hand_landmarks:
#    print('hand_landmarks:', hand_landmarks)
#    hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y